# Ray RLlib 강화학습 실험

이 노트북은 Ray RLlib을 사용한 강화학습 연구의 기본 템플릿입니다.

In [1]:
# 필수 라이브러리 import
import ray
from ray import tune
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.algorithm import Algorithm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import os

# 결과 저장 디렉토리 설정
RESULTS_DIR = Path("../results")
RESULTS_DIR.mkdir(exist_ok=True)

print("라이브러리 import 완료")

라이브러리 import 완료


## 1. Ray 초기화

In [2]:
# Ray 초기화
# 로컬 모드로 실행 (디버깅용)
# ray.init(local_mode=True)

# 일반 모드로 실행 (실험용)
ray.init(ignore_reinit_error=True)

print(f"Ray 클러스터 정보: {ray.cluster_resources()}")

2026-01-07 17:17:05,926	INFO worker.py:2023 -- Started a local Ray instance.


Ray 클러스터 정보: {'GPU': 1.0, 'node:__internal_head__': 1.0, 'memory': 89225572352.0, 'CPU': 32.0, 'accelerator_type:G': 1.0, 'node:10.254.233.114': 1.0, 'object_store_memory': 38239531008.0}


/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/_private/worker.py:2062: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(


## 2. 환경 설정

**참고**: 실제 환경은 별도 프로젝트로 관리되므로, 여기서는 예시로 Gymnasium 환경을 사용합니다.
실제 환경을 사용할 때는 해당 환경의 import 경로를 수정하세요.

In [ ]:
ENV_NAME = "horcrux_env/plane-v0"

# 환경 정보 확인
import gymnasium as gym
import horcrux_env
from horcrux_env.envs import PlaneJoyDirWorld

env = gym.make(ENV_NAME)
print(f"환경: {ENV_NAME}")
print(f"관찰 공간: {env.observation_space}")
print(f"행동 공간: {env.action_space}")
env.close()

환경: horcrux_env/plane-v0
관찰 공간: Box(-inf, inf, (97,), float32)
행동 공간: Box(0.0, 2.7, (14,), float32)


In [ ]:
ENV_CONFIG = {
    "forward_reward_weight": 175.0,
    "rotation_reward_weight": 100.0,
    "unhealthy_max_steps": 150.0,
    "healthy_reward": 3.0,
    "healthy_roll_range": (-40,40),
    "terminating_roll_range": (-85,85),
    "rotation_norm_cost_weight": 14.5,
    "termination_reward": 0,
    "gait_params": (30, 30, 40, 40, 0, -1),
    "use_friction_chg": True,
    "joy_input_random": True,
    "use_imu_window": True,
    "use_vels_window": True,
    "ctrl_cost_weight": 0.05,
}

RENDER_ENV_CONFIG = ENV_CONFIG.copy()
RENDER_ENV_CONFIG['render_mode'] = 'rgb_array'
RENDER_ENV_CONFIG['render_camera_name'] = 'ceiling'

# env = gym.make(ENV_NAME, **RENDER_ENV_CONFIG)

from ray.tune.registry import register_env
register_env(ENV_NAME, lambda config: PlaneJoyDirWorld( forward_reward_weight=ENV_CONFIG["forward_reward_weight"], 
                                                     rotation_reward_weight=ENV_CONFIG["rotation_reward_weight"], 
                                                     unhealthy_max_steps=ENV_CONFIG["unhealthy_max_steps"],
                                                     healthy_reward=ENV_CONFIG["healthy_reward"], 
                                                     healthy_roll_range=ENV_CONFIG["healthy_roll_range"],
                                                     terminating_roll_range=ENV_CONFIG["terminating_roll_range"],
                                                     rotation_norm_cost_weight=ENV_CONFIG["rotation_norm_cost_weight"],
                                                     termination_reward=ENV_CONFIG["termination_reward"],
                                                     gait_params=ENV_CONFIG["gait_params"],
                                                     use_friction_chg=ENV_CONFIG["use_friction_chg"],
                                                     joy_input_random=ENV_CONFIG["joy_input_random"],
                                                     use_imu_window=ENV_CONFIG["use_imu_window"],
                                                     ctrl_cost_weight=ENV_CONFIG["ctrl_cost_weight"],
                                                   )
            )

## 3. RLlib 알고리즘 설정

In [4]:
# PPO 알고리즘 설정 (다른 알고리즘으로 변경 가능: DQN, A3C, SAC 등)
config = (
    PPOConfig()
    .environment(env=ENV_NAME)
    .training(
        lr=3e-4,
        train_batch_size=4000,
        num_sgd_iter=10,
    )
    .resources(num_gpus=1)  # GPU 사용 시 1로 변경
    .framework("torch")  # "tf2" 또는 "torch"
    .env_runners(
        num_env_runners=16,
        num_envs_per_env_runner=8,
    )
)

print("알고리즘 설정 완료")
print(config)

2026-01-07 17:17:10,739	WARNING algorithm_config.py:2484 -- DeprecationWarning: `config.training(num_sgd_iter=..)` has been deprecated. Use `config.training(num_epochs=..)` instead. This will raise an error in the future!


알고리즘 설정 완료


## 4. 알고리즘 인스턴스 생성 및 학습

In [6]:
# 알고리즘 인스턴스 생성
algo = config.build_algo()

print("알고리즘 인스턴스 생성 완료")

/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/algorithms/algorithm.py:525: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and m

IndexError: list index out of range

In [ ]:
# 학습 실행
NUM_ITERATIONS = 100  # 학습 반복 횟수
training_history = []

for i in range(NUM_ITERATIONS):
    result = algo.train()
    training_history.append({
        'iteration': i + 1,
        'episode_reward_mean': result.get('episode_reward_mean', 0),
        'episode_len_mean': result.get('episode_len_mean', 0),
        'policy_loss': result.get('info', {}).get('learner', {}).get('default_policy', {}).get('policy_loss', 0),
    })
    
    if (i + 1) % 10 == 0:
        print(f"Iteration {i + 1}: Mean Reward = {result.get('episode_reward_mean', 0):.2f}")

print("학습 완료")

## 5. 학습 결과 시각화

In [ ]:
# 학습 히스토리를 DataFrame으로 변환
df = pd.DataFrame(training_history)

# 결과 저장
df.to_csv(RESULTS_DIR / "training_history.csv", index=False)
print(f"결과 저장 완료: {RESULTS_DIR / 'training_history.csv'}")

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# 에피소드 평균 보상
axes[0].plot(df['iteration'], df['episode_reward_mean'])
axes[0].set_xlabel('Iteration')
axes[0].set_ylabel('Mean Episode Reward')
axes[0].set_title('Training Progress: Mean Reward')
axes[0].grid(True)

# 에피소드 평균 길이
axes[1].plot(df['iteration'], df['episode_len_mean'])
axes[1].set_xlabel('Iteration')
axes[1].set_ylabel('Mean Episode Length')
axes[1].set_title('Training Progress: Mean Episode Length')
axes[1].grid(True)

plt.tight_layout()
plt.savefig(RESULTS_DIR / "training_curves.png", dpi=150)
plt.show()

print(f"시각화 저장 완료: {RESULTS_DIR / 'training_curves.png'}")

In [ ]:
# 학습된 정책으로 평가
NUM_EVAL_EPISODES = 10
eval_results = []

env = gym.make(ENV_NAME, render_mode=None)

for episode in range(NUM_EVAL_EPISODES):
    obs, info = env.reset()
    episode_reward = 0
    episode_length = 0
    done = False
    
    while not done:
        action = algo.compute_single_action(obs)
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        episode_reward += reward
        episode_length += 1
    
    eval_results.append({
        'episode': episode + 1,
        'reward': episode_reward,
        'length': episode_length
    })

env.close()

eval_df = pd.DataFrame(eval_results)
print("\n평가 결과:")
print(eval_df)
print(f"\n평균 보상: {eval_df['reward'].mean():.2f} ± {eval_df['reward'].std():.2f}")
print(f"평균 에피소드 길이: {eval_df['length'].mean():.2f} ± {eval_df['length'].std():.2f}")

# 평가 결과 저장
eval_df.to_csv(RESULTS_DIR / "evaluation_results.csv", index=False)

## 7. 모델 저장 및 로드

In [ ]:
# 모델 저장
checkpoint_dir = algo.save(RESULTS_DIR / "checkpoint")
print(f"모델 저장 완료: {checkpoint_dir}")

# 모델 로드 예시 (주석 처리)
# loaded_algo = Algorithm.from_checkpoint(checkpoint_dir)

## 8. 정리

In [ ]:
# 알고리즘 정리
algo.stop()

# Ray 종료 (필요한 경우)
# ray.shutdown()

print("정리 완료")